### 타깃
[naver news:공공자전거](https://search.naver.com/search.naver?&where=news&query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&start=1&refresh_start=0)

#### Ref
- [selenium attribute](https://stackoverflow.com/questions/30324760/how-to-get-attribute-of-element-from-selenium?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)
- [~~selenium multiple class~~](https://stackoverflow.com/questions/21713280/find-div-element-by-multiple-class-names?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa) selenium옛버전. 이젠 contains 나 findElement는 안됨

In [132]:
# coding: utf-8
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os.path
from os import rename
from os import listdir

import time

import csv

In [133]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import re

In [110]:
def startWeb(numPage=1):
    """
    네이버 뉴스
    검색어: 공공자전거
    옵션유지: 켜짐. mynews=1
    @return driver
    """
    driver= webdriver.Chrome('lib/chromedriver.exe')
    paging= str(1+10*(numPage-1))
    # caution: "\ is before &"
    url= "https://search.naver.com/search.naver?&where=news\
    &query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge\
    &sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=\
    &nso=so:r,p:all,a:all&mynews=1&start="+paging+"&refresh_start=0"
    driver.get(url)
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//li'))
        )
    except Exception as e:
        print('error',e)
    return driver

#### 언론사 선택

In [111]:
def searchOptPress():
    """
    일간지, 뉴시스, 연합뉴스, 한국경제TV, JTBC, 시사IN
    """
    #검색옵션
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//a[@id="_search_option_btn"]'))
        )
    except Exception as e:
        print('error',e)
    element.click()

    #언론사
    xpath1= '//li[@id="news_popup"]'
    element= driver.find_element_by_xpath(xpath1)
    element.click()

    # #전체선택 해제. '18.4.24 부로 전체선택옵션이 없어졌다 ㄷㄷ
    # xpath2= '//input[@id="select_all_order"]'
    # element= driver.find_element_by_xpath(xpath2)
    # element.click()

    #일간지 선택
    xpath3= '//input[@id="ca_p1"]'
    element= driver.find_element_by_xpath(xpath3)
    element.click()
    
    somePress= ["뉴시스", "연합뉴스", "한국경제TV", "JTBC", "시사IN"]
    for press in somePress:
        xpath= '//label[@title="'+press+'"]'
        element= driver.find_element_by_xpath(xpath)
        element.click()
        
    # 확인
    xpathChk= '//button[@class="impact _submit_btn"]'
    element= driver.find_element_by_xpath(xpathChk)
    element.click()
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//li'))
        )
    except Exception as e:
        print('error',e)

#### 링크 얻기

In [112]:
def movePage(numPage):
    paging= str(1+10*(numPage-1))
    url= "https://search.naver.com/search.naver?&where=news\
    &query=%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0&sm=tab_pge\
    &sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=\
    &nso=so:r,p:all,a:all&mynews=1&start="+paging+"&refresh_start=0"
    driver.get(url)
    
    try:
        element= WebDriverWait(driver, 10).until(
            # is comming content?
            EC.presence_of_element_located((By.XPATH, '//div[@class="thumb"]'))
        )
    except Exception as e:
        print('error',e)

In [113]:
def saveLink(listLink):
    #caution: not '_sp_each_title' but ' _sp_each_title'
    #amazing: there are no duplicated. why ??
    
    xpathLink1= '//a[@class=" _sp_each_title"]'
    elements= driver.find_elements_by_xpath(xpathLink1)
    for element in elements:
        listLink.append(element.get_attribute("href"))
        
    xpathLink2= '//a[@class="_sp_each_url _sp_each_title"]'
    elements= driver.find_elements_by_xpath(xpathLink2)
    for element in elements:
        listLink.append(element.get_attribute("href"))
        
    return listLink

#### 각 링크의 내용

In [114]:
def movePageLink(link):
    driver.get(link)
    try:
        element= WebDriverWait(driver, 10).until(
            # is coming a content?
            EC.presence_of_element_located((By.XPATH, '//div'))
        )
    except Exception as e:
        print('error',e)

#### 전제(낙관)
같은 종류의 사이트라면 같은 xpath를 쓸 것이다

In [128]:
def mkDictNewsXpath():
    """
    link scraping 결과에 unique() 하고, 의뭉스런 사이트를 직접 들어가서
    해당 언론사가 필터링한 언론사 맞는지 확인한 결과
    
    xpath가 틀렸다면 여기서 정정!
    """
    link= ['news.khan.co', 'www.khan.co', 'h2.khan.co', 'news.kmib.co', 'www.naeil.com', 'news.donga.com', 'www.donga.com', 'realestate.donga.com', 'bizn.donga.com', 'travel.donga.com', 'studio.donga.com', 'yachtpia.donga.com', 'it.donga.com', 'www.m-i.kr', 'www.munhwa.com', 'www.seoul.co', 'ntn.seoul.co', 'stv.seoul.co', 'www.segye.com', 'local.segye.com', 'www.segyefn.com', 'economysegye.segye.com', 'fn.segye.com', 'www.asiatoday.co', 'news.chosun.com', 'biz.chosun.com', 'car.chosun.com', 'life.chosun.com', 'businessnews.chosun.com', 'news.joins.com', 'realestate.joins.com', 'www.hani.co', 'babytree.hani.co', '2korea.hani.co', 'www.hankookilbo.com', 'www.newsis.com', 'sports.news.naver', 'news.naver.com', 'app.yonhapnews.co', 'www.wowtv.co', 'www.wownet.co', 'news.wowtv.co', 'sports.wowtv.co', 'wowstar.wowtv.co', 'news.jtbc.co', 'www.sisain.co', 'www.sisainlive.com']
    dupCnt= [3,1,1,8,1,1,3,5,1,5,2,3,1,2,2,5,1,2]
    xpaths= ['//div[@id="articleBody"]', '//div[@id="articleBody"]', '//div[@class="articleArea"]', '//div[@id="ct"]', '//div[@class="content"]', '//div[@id="NewsAdContent"]', '//div[@id="article_content"]', '//div[@id="article_txt"]', '//div[@id="font"]', '//div[@id="news_body_id"]/div[@class="par"]', '//div[@id="article_body"]', '//div[@class="article-text-font-size"]', '//article[@id="article-body"]','//div[@itemprop="articleBody"]', '//div[@id="articleWrap"]', '//div[@id="divNewsContent"]', '//div[@class="article_content"]', '//div[@itemprop="articleBody"]']
    xpaths= np.array(xpaths)

    xpaths_s= list(xpaths.repeat(dupCnt))

    dictLink= dict(zip(link,xpaths_s))
    return dictLink

In [129]:
dictLink= mkDictNewsXpath()
def whatLinksXpath(link):
    link3piece= '.'.join(link.split('/')[2].split('.')[0:3])
    xpath= dictLink[link3piece]
    return xpath

In [117]:
def rmSpecialChr(weirdStr):
    """
    @param string
    @return string. #특문제거, \n -> ___
    """
    regPattLine= '\n'
    res= re.sub(regPattLine,'___',weirdStr)
    regPatt= '[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9()": ]|[[\u3131-\u318E\uAC00-\uD7A3]]'
    res= re.sub(regPatt,'',res)
    res= re.sub('\\s+', ' ', res) # 공백 제거 해,말아? 하자. 안 하면 헬일듯
    return res

In [171]:
def getContent(link):
    """
    @return conBodyText
    """
    xpathBody= whatLinksXpath(link)
    #print(xpathBody)
    
    try:
        conBody= WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, xpathBody))
        )
        conBodyText= rmSpecialChr(conBody.text)
    except Exception as e:
        print('no match xpath',e)
        conBodyText= ""
        pass
    
    return conBodyText

In [119]:
driver.close()
driver.quit()

###### 세션 종료, 자원반납

In [127]:
driver.close()
driver.quit()

## test_성공
1페이지에서 link 목록 따오기

In [150]:
# WELL-DONE !
driver= startWeb()

In [151]:
searchOptPress()

In [152]:
numPage= 1
movePage(numPage)

In [155]:
listLink= list([])
listLink= saveLink(listLink)

listLink

['http://www.newsis.com/view/?id=NISX20180421_0000288016&cID=10803&pID=10800',
 'http://www.seoul.co.kr/news/newsView.php?id=20180417014003&wlog_tag3=naver',
 'http://www.hankookilbo.com/v/0e2f312b60b84e099b87d04436fc3308',
 'http://bizn.donga.com/realestate/3/all/20180416/89637182/2',
 'http://news.joins.com/article/olink/22127647',
 'http://news.joins.com/article/olink/22120203',
 'http://news.chosun.com/site/data/html_dir/2018/04/10/2018041000149.html',
 'http://app.yonhapnews.co.kr/YNA/Basic/SNS/r.aspx?c=AKR20180408025600004&did=1195m',
 'http://www.asiatoday.co.kr/view.php?key=20180422010012584',
 'http://www.asiatoday.co.kr/view.php?key=20180418010010605']

## test _성공
link 하나에서 뉴스 내용 퍼오기

In [120]:
# WELL-DONE !
driver= startWeb()

In [121]:
searchOptPress()

In [122]:
# numPage= 2
# movePage(numPage)

In [123]:
dictLink= mkDictNewsXpath() # 최초 1번만 만들면 된다

linkEx= 'http://www.newsis.com/view/?id=NISX20180421_0000288016&cID=10803&pID=10800'

In [124]:
movePageLink(linkEx)

In [125]:
contents= getContent(linkEx)
contents

//div[@itemprop="articleBody"]


'수원뉴시스김경호 기자 수원시 무인 대여자전거수원뉴시스김경호 기자 경기 수원시가 지난해 전국 최초로 도입한 스테이션 없는 무인대여자전거가 올해 상반기 중 대폭 늘어난다 수원시는 무인대여자전거 950대를 운영하는 공유자전거 업체 모바이크(Mobike)가 상반기 안에 4050대를 추가한다고 21일 밝혔다 5월 중 1400대 6월 중 2650대를 배치할 계획이다 새롭게 배치되는 자전거는 뉴라이트 모델로 기존 자전거보다 성능이 개선됐다 자전거 무게를 6(2216) 줄였고 잠금장치 배터리 방전을 방지하기 위한 태양광 패널을 부착했다 여성아동도 쉽게 탈 수 있도록 안장 높이 조절 범위를 늘렸다 회원 가입할 때 내야 했던 보증금(5000원)은 23일부터 없애 이용자 부담을 줄인다 또 국내 모든 신용카드로 이용료를 결제할 수 있도록 결제 시스템을 개선할 계획이다 수원시는 5월부터 무인대여 자전거 주차구역 관리(자전거 정리정돈거치대 청소방치자전거 처리 등)를 민간기관에 위탁해 시민 불편을 최소화할 계획이다 자전거 이용경로를 알려주는 빅데이터를 활용해 요소요소에 자전거 주차장을 확보하고 자전거도로 정비 우선순위를 정할 예정이다 무인대여자전거 시스템은 자전거 거치대 무인 정보안내시스템 등이 필요 없다 스마트폰을 활용해 GPS가 장착된 자전거를 수원시 곳곳에 있는 자전거 주차공간에서 간편하게 대여하고 반납할 수 있다 현재 수원시에서는 공유자전거 업체인 모바이크와 오바이크(oBike)가 무인대여 자전거를 운영하고 있다 무인대여 자전거를 이용하려면 먼저 스마트폰 플레이스토어(안드로이드폰) 앱스토어(아이폰)에서 오바이크나 모바이크를 검색해 애플리케이션을 내려받아야 한다 무인자전거 앱을 활성화하면 주변에 있는 자전거 위치를 알려주는 지도가 나온다 자전거가 있는 장소를 찾아가 앱 하단 잠금 해제를 누른 후 스마트폰으로 자전거 핸들 사이에 부착된 큐알(QR)코드를 스캔하면 잠금이 해제된다 자전거를 타고 목적지까지 이동한 뒤 목적지 주변에 있는 공공자전거 주차공간(노면에 표시)이나 자전거 거치

---

# 할일: 링크 목록 돌려서 df 에 차곡차곡 쌓기

---

## 실행- 링크얻기_완료
링크가 필요하면 또 돌리되, 시간이 제법 걸린다

In [158]:
# driver= startWeb()
# searchOptPress()

In [131]:
## data/naverNewsSomePressLinks.csv 에 저장 완료!
# numPages= range(1,400)
# listLink= list([])

# for numPage in numPages:
#     print(numPage)
#     movePage(numPage)
#     listLink= saveLink(listLink)

#print(len(listLink))
# listLink

In [75]:
# driver.close()
# driver.quit()

# 저장&읽기

### 저장 from Scraping
- [dictionary to csv](https://stackoverflow.com/questions/8685809/python-writing-a-dictionary-to-a-csv-file-with-one-line-for-every-key-value?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)
    - [writerow one line feed](https://stackoverflow.com/questions/3191528/csv-in-python-adding-an-extra-carriage-return?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)

In [130]:
# # scraping 직후. 옵션외 링크가 좀 섞여있다
# DataFrame(listLink).to_csv("data/naverNewsSomePressLinks.csv", index= False, header=None)

#### 읽기
scraping 직후. 옵션외 링크가 좀 섞여있다

In [19]:
# linkRead= pd.read_csv("data/naverNewsSomePressLinks.csv", header=None)

# linkRead= list(linkRead[0])

# print(len(linkRead))

# linkRead[30:35]

#### 옵션 외 언론사 지우기
133개 지움

In [134]:
# df= DataFrame(linkRead)

# exceptList= ['www.kukinews.com', 'news.kukinews.com', 'www.seouland.com', 'nownews.seoul.co', 'nownewstv.seoul.co', 'www.sporbiz.co', 'www.beautyhankook.com']
# for exLink in exceptList:
#     df= df[df[0].map(lambda x: (exLink not in x))]
    
# print(len(df))

# 다시 저장 though exceptList
이게 제대로된 링크 목록

In [26]:
df.to_csv("data/naverNewsDailyPlusPressLinks.csv", index= False, header=None)

### 읽기
광고? 제거된 링크

In [139]:
linkNewsDailyPlus= pd.read_csv('data/naverNewsDailyPlusPressLinks.csv',
                               header=None)

print(linkNewsDailyPlus.shape)
print(type(linkNewsDailyPlus[0]))
linkNewsDailyPlus.head(3)

(3857, 1)
<class 'pandas.core.series.Series'>


,0
0,http://www.newsis.com/view/?id=NISX20180421_00...
1,http://www.seoul.co.kr/news/newsView.php?id=20...
2,http://www.hankookilbo.com/v/0e2f312b60b84e099...


In [141]:
linkNewsDailyPlus_list= list(linkNewsDailyPlus[0])
len(linkNewsDailyPlus_list)

3857

# 실행- 내용얻기

In [173]:
driver.close()
driver.quit()

#### 링크에서 각 값 따오기-> DataFrame 만들기

In [174]:
linkNewsDailyPlus= pd.read_csv('data/naverNewsDailyPlusPressLinks.csv',
                               header=None)
linkNewsDailyPlus_list= list(linkNewsDailyPlus[0])
print(len(linkNewsDailyPlus_list))

bodyList= list([])
pressList3pie= list([])

dictLink= mkDictNewsXpath() # 최초 1번만 만들면 된다. driver와 무관

3857


In [175]:
driver= startWeb()
searchOptPress()

for i, link in enumerate(linkNewsDailyPlus_list):
    #if (0== i)| (1==i):
    #print(link)
    print(i)

    movePageLink(link)
    body= getContent(link)
    bodyList.append(body)

    link3piece= '.'.join(link.split('/')[2].split('.')[0:3])
    pressList3pie.append(link3piece)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
no match xpath Message: 

18
19
20
21
22
23
no match xpath Message: 

24
25
26
27
no match xpath Message: 

28
29
no match xpath Message: 

30
31
32
33
34
35
no match xpath Message: 

36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
no match xpath Message: 

53
54
55
56
57
58
59
60
61
no match xpath Message: 

62
63
64
65
no match xpath Message: 

66
67
68
no match xpath Message: 

69
70
71
72
73
74
75
76
no match xpath Message: 

77
no match xpath Message: 

78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
no match xpath Message: 

96
97
no match xpath Message: 

98
no match xpath Message: 

99
100
101
102
103
104
105
106
107
108
109
110
111
no match xpath Message: 

112
no match xpath Message: 

113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
no match xpath Message: 

134
135
136
no match xpath Message: 

137
no match xpath Message: 

138
139
140
141
142
143
144
no match xpath Message: 

145
146
no

971
no match xpath Message: 

972
no match xpath Message: 

973
974
no match xpath Message: 

975
976
977
978
979
980
no match xpath Message: 

981
982
no match xpath Message: 

983
984
985
no match xpath Message: 

986
987
988
989
990
no match xpath Message: 

991
no match xpath Message: 

992
993
994
995
996
no match xpath Message: 

997
998
999
1000
1001
no match xpath Message: 

1002
no match xpath Message: 

1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
no match xpath Message: 

1025
1026
1027
no match xpath Message: 

1028
1029
1030
1031
no match xpath Message: 

1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
no match xpath Message: 

1045
1046
1047
1048
1049
1050
1051
no match xpath Message: 

1052
1053
1054
1055
1056
no match xpath Message: 

1057
1058
1059
1060
1061
1062
1063
1064
no match xpath Message: 

1065
1066
no match xpath Message: 

1067
1068
1069
1070
1071


KeyboardInterrupt: 

In [176]:
newsData= { 
    'link3pie': pressList3pie,
    'zbody':bodyList
}
dfNews= pd.DataFrame(data= newsData)

### 저장

In [177]:
dfNews.to_csv('data/naverNewsContent.csv', index= False, encoding= 'cp949')

In [179]:
dfNews['link3pie'].value_counts()

app.yonhapnews.co          204
www.newsis.com             181
www.asiatoday.co           113
www.m-i.kr                  83
news.khan.co                60
www.hankookilbo.com         54
www.segye.com               48
www.seoul.co                45
news.joins.com              41
news.donga.com              39
news.kmib.co                32
www.hani.co                 32
www.munhwa.com              31
www.naeil.com               24
news.chosun.com             21
news.jtbc.co                19
news.naver.com              16
news.wowtv.co                6
bizn.donga.com               5
www.sisain.co                3
www.wowtv.co                 3
babytree.hani.co             2
www.khan.co                  2
realestate.joins.com         2
travel.donga.com             1
stv.seoul.co                 1
car.chosun.com               1
biz.chosun.com               1
businessnews.chosun.com      1
Name: link3pie, dtype: int64

In [1]:
# dfNotice['writerP'].value_counts()

In [2]:
import pandas as pd

## 읽기

In [3]:
df= pd.read_csv("data/naverNewsContent.csv", encoding= 'cp949')

In [4]:
df.columns

Index(['date', 'title', 'writerP', 'zbody'], dtype='object')

#### 문자열 포함된 거 찾기

In [9]:
def findStr(inputStr):
    tradein= lambda x:inputStr in x
    return df[df.zbody.map(tradein)]

In [17]:
inputStr= '음식'
findStr(inputStr).to_csv('data/newsfind'+inputStr+'.csv', index= False, encoding= 'cp949')

---

---

---

#### 어느 사이트인지 판단
18개 xpath 중에 무엇일지

In [27]:
# linkRead= pd.read_csv("data/naverNewsDailyPlusPressLinks.csv", header=None)
# linkRead= list(linkRead[0])

# pressList3piece= ['.'.join(x.split('/')[2].split('.')[0:3]) for x in linkRead]

# print(Series(pressList3piece).unique())
# print(len(Series(pressList3piece).unique()))
# print(Series(pressList3piece).value_counts())

# pressList2piece= ['.'.join(x.split('/')[2].split('.')[1:3]) for x in linkRead]

# print(Series(pressList2piece).unique())
# print(len(Series(pressList2piece).unique()))
# print(Series(pressList2piece).value_counts())

# # 18
# # 개 언론사만 골라서했는데 27개 나옴. > 무슨무슨링크랑 회사랑 매치되는지 알아야
# # - news.naver.com : 연합뉴스
# # - sports.news.naver : 뉴시스

# dfLinkRead= DataFrame(linkRead)
# dfLinkEx= dfLinkRead[dfLinkRead[0].map(lambda x:'naver.com' in x)]
# print(dfLinkEx)

# # sports.news.naver  3
# # news.naver.com 928
# print(len(dfLinkEx))

In [34]:
# # naver 있는 게 왜이리 많은지 보기
# dfLinkEx.to_csv("data/naver_in_link.csv", index= False, header=None)

#### link~xpath Dictionary 만드는 코드 백업

In [41]:
# df= pd.read_csv('data/naverNewsSomePressLinks.csv', index_col=False, header=None)
#len(df)

# ## 긁으려는 거 외 링크 지우기
# # 133개 지움
# exceptList= ['www.kukinews.com', 'news.kukinews.com', 'www.seouland.com', 'nownews.seoul.co', 'nownewstv.seoul.co', 'www.sporbiz.co', 'www.beautyhankook.com']
# for exLink in exceptList:
#     df= df[df[0].map(lambda x: (exLink not in x))]

#df
#len(df)
#df[0].map(lambda x: (exceptList[0] in x)).sum()

## dict 만들기
# key: http빼고 사이트 앞 3어절, value: XPATH
# link= ['news.khan.co', 'www.khan.co', 'h2.khan.co', 'news.kmib.co', 'www.naeil.com', 'news.donga.com', 'www.donga.com', 'realestate.donga.com', 'bizn.donga.com', 'travel.donga.com', 'studio.donga.com', 'yachtpia.donga.com', 'it.donga.com', 'www.m-i.kr', 'www.munhwa.com', 'www.seoul.co', 'ntn.seoul.co', 'stv.seoul.co', 'www.segye.com', 'local.segye.com', 'www.segyefn.com', 'economysegye.segye.com', 'fn.segye.com', 'www.asiatoday.co', 'news.chosun.com', 'biz.chosun.com', 'car.chosun.com', 'life.chosun.com', 'businessnews.chosun.com', 'news.joins.com', 'realestate.joins.com', 'www.hani.co', 'babytree.hani.co', '2korea.hani.co', 'www.hankookilbo.com', 'www.newsis.com', 'sports.news.naver', 'news.naver.com', 'app.yonhapnews.co', 'www.wowtv.co', 'www.wownet.co', 'news.wowtv.co', 'sports.wowtv.co', 'wowstar.wowtv.co', 'news.jtbc.co', 'www.sisain.co', 'www.sisainlive.com']
# dupCnt= [3,1,1,8,1,1,3,5,1,5,2,3,1,2,2,5,1,2]
# xpaths= ['//div[@id="articleBody"]', '//div[@id="articleBody"]', '//div[@class="articleArea"]', '//div[@id="ct"]', '//div[@class="content"]', '//div[@id="NewsAdContent"]', '//div[@id="article_content"]', '//div[@id="article_txt"]', '//div[@id="font"]', '//div[@id="news_body_id"]/div[@class="par"]', '//div[@id="article_body"]', '//div[@class="article-text-font-size"]', '//article[@id="article-body"]', '//div[@id="#textBody"]', '//div[@id="articleWrap"]', '//div[@id="divNewsContent"]', '//div[@class="article_content"]', '//div[@itemprop="articleBody"]']
# xpaths= np.array(xpaths)
# print('xpaths렝',len(xpaths))
# print('dupCnt렝',len(dupCnt))

# xpaths_s= list(xpaths.repeat(dupCnt))
# xpaths_s
# print('link>> ',link)
# print('xpaths_s>> ',xpaths_s)

# dictLink= dict(zip(link,xpaths_s))
#dictLink

# ## xpath 얻기
# sriLink= df[0].map(lambda x: '.'.join(x.split('/')[2].split('.')[0:3]))
# #sriLink
# #sriLink.value_counts()

# ## 미리 만들어둔 list ( link ~ xpath ) ㄴㄴ
# xpathList= list([])
# for link in sriLink:
#     xpathList.append(dictLink[link])
# xpathList